# Week 3: Feature Engineering
## Extracting Features from ECG (RR Intervals) and SpO2 Signals

**Objective**: Calculate Time-Domain, Frequency-Domain, Non-Linear, and SpO2 features for Sleep Apnea Detection.

### Feature Sets:
1. **Time-Domain**: Mean RR, SDNN, RMSSD, pNN50
2. **Frequency-Domain**: Avg Power (LF, HF), LF/HF Ratio
3. **Non-Linear**: SD1, SD2 (Poincare Plot)
4. **SpO2 Features**: Mean Saturation, ODI (Dips > 3%)

### Pipeline:
1. Load Data (from .mat files)
2. Preprocess (Filter Artifacts)
3. Extract Features per Segment
4. Aggregate into Feature Matrix (X, y)

In [1]:
import numpy as np
import pandas as pd
import scipy.io
import scipy.signal
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully")

Libraries imported successfully


In [2]:
# Configuration
DATA_PATH = '../APNEA HRV+SPO2 DATASET/HuGCDN2014-OXI'
FOLDERS = {'C': 'C', 'D': 'D', 'ND': 'ND'}
FS_RR = 1  # RR intervals are essentially discrete events, but derived from freq
       # Actually, for spectral analysis of RR series, we usually resample to e.g. 4Hz

def load_mat_data(filename, folder, key_hint):
    # Handle potential subfolders 'RR' and 'SAT' which seem to separate data types
    # Inspecting file structure showed folders C, D, ND directly under data path
    # But script showed sample files in subdirs. Let's try flexible loading.
    
    # Actually list_dir output showed:
    # DATA_PATH/RR/C1.mat etc
    # DATA_PATH/SAT/C1.mat etc
    # So structure is likely: TYPE/Filename
    
    # Adjust path logic based on inspection
    # We need to construct path: DATA_PATH / TYPE / FILENAME
    # filename e.g. 'C1.mat'
    # folder (label) is inferred from filename prefix
    
    if 'RR' in key_hint:
        sub = 'RR'
    else:
        sub = 'SAT'
        
    path = os.path.join(DATA_PATH, sub, filename)
    
    if not os.path.exists(path):
        # Fallback to direct folder if flattened
        path = os.path.join(DATA_PATH, folder, filename)

    try:
        mat = scipy.io.loadmat(path)
        keys = [k for k in mat.keys() if not k.startswith('__')]
        for k in keys:
            if key_hint in k:
                return mat[k]
        # Fallback: return first key if specific hint not found
        return mat[keys[0]] if keys else None
    except Exception as e:
        # print(f"Error loading {path}: {e}")
        return None

## 1. Time-Domain Features
- **Mean RR**: Average time between beats.
- **SDNN**: Standard deviation of NN (RR) intervals.
- **RMSSD**: Root mean square of successive differences.
- **pNN50**: Percentage of successive RR intervals differing by > 50ms.

In [3]:
def get_time_domain_features(rr_intervals):
    if len(rr_intervals) < 2:
        return [np.nan] * 4
    
    diff_rr = np.diff(rr_intervals)
    
    mean_rr = np.mean(rr_intervals)
    sdnn = np.std(rr_intervals, ddof=1)
    rmssd = np.sqrt(np.mean(diff_rr ** 2))
    pnn50 = np.sum(np.abs(diff_rr) > 50) / len(diff_rr) * 100
    
    return [mean_rr, sdnn, rmssd, pnn50]

## 2. Frequency-Domain Features
Using Welch's method to estimate Power Spectral Density (PSD).
- **VLF**: Very Low Frequency (0.0033 - 0.04 Hz)
- **LF**: Low Frequency (0.04 - 0.15 Hz)
- **HF**: High Frequency (0.15 - 0.4 Hz)
- **LF/HF Ratio**: Sympathovagal balance

In [4]:
from scipy.interpolate import interp1d

def get_freq_domain_features(rr_intervals):
    if len(rr_intervals) < 2:
        return [np.nan] * 4
        
    # Resample RR intervals to 4Hz for uniform sampling (required for FFT/Welch)
    # Create time axis
    time_cumsum = np.cumsum(rr_intervals) / 1000.0
    time_cumsum = time_cumsum - time_cumsum[0]
    
    # Interpolation function
    f_interp = interp1d(time_cumsum, rr_intervals, kind='cubic', fill_value="extrapolate")
    
    # New time axis (4Hz)
    fs = 4.0
    steps = 1 / fs
    new_time = np.arange(0, time_cumsum[-1], steps)
    rr_interpolated = f_interp(new_time)
    
    # Detrend
    rr_detrended = scipy.signal.detrend(rr_interpolated)
    
    # Welch's Periodogram
    f, Pxx = scipy.signal.welch(rr_detrended, fs=fs, nperseg=256)
    
    # Band Power
    # VLF: 0.0033 - 0.04 Hz
    # LF: 0.04 - 0.15 Hz
    # HF: 0.15 - 0.4 Hz
    
    vlf_mask = (f >= 0.0033) & (f < 0.04)
    lf_mask = (f >= 0.04) & (f < 0.15)
    hf_mask = (f >= 0.15) & (f < 0.4)
    
    vlf_power = np.trapz(Pxx[vlf_mask], f[vlf_mask])
    lf_power = np.trapz(Pxx[lf_mask], f[lf_mask])
    hf_power = np.trapz(Pxx[hf_mask], f[hf_mask])
    
    lf_hf_ratio = lf_power / hf_power if hf_power > 0 else 0
    
    return [vlf_power, lf_power, hf_power, lf_hf_ratio]

## 3. Non-Linear Features
Poincare Plot Geometry (SD1, SD2).
- **SD1**: Short-term variability (width of ellipse).
- **SD2**: Long-term variability (length of ellipse).

In [5]:
def get_nonlinear_features(rr_intervals):
    if len(rr_intervals) < 2:
        return [np.nan] * 2
        
    # RR(n) vs RR(n+1)
    x = rr_intervals[:-1]
    y = rr_intervals[1:]
    
    # SD1, SD2 calculation
    sd1 = np.std(np.subtract(x, y) / np.sqrt(2))
    sd2 = np.std(np.add(x, y) / np.sqrt(2))
    
    return [sd1, sd2]

## 4. SpO2 Features
- **Mean SpO2**: Average oxygen saturation.
- **Min SpO2**: Minimum value.
- **ODI (Oxygen Desaturation Index)**: Number of times SpO2 drops by > 3% from baseline (approx).

In [6]:
def get_spo2_features(spo2_signal):
    if len(spo2_signal) == 0:
        return [np.nan] * 3
        
    mean_sat = np.mean(spo2_signal)
    min_sat = np.min(spo2_signal)
    
    # Simple ODI calculation: Count drops > 3%
    # 1. Start from baseline (e.g., median)
    # 2. Find local dips
    # This is a simplified version. 
    
    # Calculate difference from baseline
    baseline = np.median(spo2_signal)
    
    # Count desaturation events (simplified)
    desats = 0
    in_event = False
    for val in spo2_signal:
        if val < (baseline - 3):
            if not in_event:
                desats += 1
                in_event = True
        else:
            in_event = False
            
    # ODI is events per hour. 
    # Assuming signal duration is known or calculated from length / sampling_rate
    # Here we just return raw count for the segment, normalization happens later if segment time is known
    
    return [mean_sat, min_sat, desats]

## 5. Main Processing Loop
Iterate through all files, clean segments, extract features, and build the dataset.

In [7]:
# This cell will take time to run
features_list = []

# Iterate over 'RR' folder since paired files exist in 'SAT'
rr_base_path = os.path.join(DATA_PATH, 'RR')

if os.path.exists(rr_base_path):
    files = [f for f in os.listdir(rr_base_path) if f.endswith('.mat')]
    
    for i, filename in enumerate(files):
        # Infer label from filename
        label = 'ND' if filename.startswith('ND') else ('D' if filename.startswith('D') else 'C')
        
        # Load data
        rr_data = load_mat_data(filename, label, 'RR')
        spo2_data = load_mat_data(filename, label, 'SAT')
        
        # Handle segmentation (assuming cleaned)
        # For this dataset, data is often split into segments within the file or just one long recording
        # We will treat the whole file as one recording for simplicity or iterate if cells
        
        # Flatten RR
        full_rr = np.array([])
        if rr_data is not None:
            if rr_data.dtype == 'O':
                valid = [c.flatten() for c in rr_data.flatten() if c.size > 0]
                if valid: full_rr = np.concatenate(valid)
            else:
                full_rr = rr_data.flatten()
        
        # Flatten SpO2
        full_spo2 = np.array([])
        if spo2_data is not None:
             if spo2_data.dtype == 'O':
                valid = [c.flatten() for c in spo2_data.flatten() if c.size > 0]
                if valid: full_spo2 = np.concatenate(valid)
             else:
                full_spo2 = spo2_data.flatten()
                
        # Skip empty
        if full_rr.size < 100 or full_spo2.size < 100:
            continue
            
        # --- Cleaning ---
        # RR: 300 - 2000 ms
        mask_rr = (full_rr >= 300) & (full_rr <= 2000)
        clean_rr = full_rr[mask_rr]
        
        # SpO2: 50 - 100 %
        mask_spo2 = (full_spo2 >= 50) & (full_spo2 <= 100)
        clean_spo2 = full_spo2[mask_spo2]
        
        if clean_rr.size < 100:
             continue

        # --- Feature Extraction ---
        # Time-Domain
        td = get_time_domain_features(clean_rr)
        
        # Frequency-Domain
        fd = get_freq_domain_features(clean_rr)
        
        # Non-Linear
        nl = get_nonlinear_features(clean_rr)
        
        # SpO2
        sp = get_spo2_features(clean_spo2)
        
        # Compile
        feat_row = {
            'filename': filename,
            'label': label,
            'mean_rr': td[0], 'sdnn': td[1], 'rmssd': td[2], 'pnn50': td[3],
            'vlf_power': fd[0], 'lf_power': fd[1], 'hf_power': fd[2], 'lf_hf_ratio': fd[3],
            'sd1': nl[0], 'sd2': nl[1],
            'mean_spo2': sp[0], 'min_spo2': sp[1], 'odi_count': sp[2]
        }
        
        features_list.append(feat_row)

# Create DataFrame
if not features_list:
    print("No features extracted. Check data path or loading logic.")
    df_features = pd.DataFrame(columns=['filename', 'label', 'mean_rr', 'sdnn', 'rmssd', 'pnn50', 'vlf_power', 'lf_power', 'hf_power', 'lf_hf_ratio', 'sd1', 'sd2', 'mean_spo2', 'min_spo2', 'odi_count'])
else:
    df_features = pd.DataFrame(features_list)
    print("Feature Extraction Complete")
    if 'label' in df_features.columns:
        print(df_features.groupby('label').size())
    else:
        print("Label column missing from DataFrame")
df_features.head()

Feature Extraction Complete
label
C     38
D     34
ND    11
dtype: int64


,filename,label,mean_rr,sdnn,rmssd,pnn50,vlf_power,lf_power,hf_power,lf_hf_ratio,sd1,sd2,mean_spo2,min_spo2,odi_count
0,C1.mat,C,913.388545,94.160944,14.975915,0.559917,327.971893,323.908679,55.926886,5.791645,10.589571,132.741503,96.025473,91.979858,1076
1,C10.mat,C,979.519626,118.289132,29.716227,7.568275,510.068515,820.147754,245.433851,3.341624,21.012545,165.960066,96.480269,90.989548,646
2,C11.mat,C,704.911414,52.355440,13.405367,0.545485,188.951448,151.283596,60.708957,2.491949,9.479026,73.432123,96.980014,80.549325,4389
3,C12.mat,C,856.033064,72.529749,16.815397,1.196960,248.683406,305.093759,82.317545,3.706303,11.890281,101.880023,95.653068,74.338903,2110
4,C13.mat,C,989.164798,109.755166,44.184377,20.621179,823.901691,843.240759,575.640730,1.464873,31.243072,152.040033,97.470270,61.039139,144


In [8]:
# Save to CSV
output_dir = '../processed_data'
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, 'extracted_features.csv')
df_features.to_csv(output_path, index=False)
print(f"Features saved to {output_path}")

Features saved to ../processed_data/extracted_features.csv
